In [ ]:
DAYS=30

In [ ]:
%run ../pathutils.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run health.ipynb
%run database.ipynb
%run utils.ipynb

In [ ]:
# Load the readings for each sensor and produce a combined data frame
bme280_df = load_sensor_readings("bme280", DAYS)
veml7700_df = load_sensor_readings("veml7700", DAYS)
sgp40_df = load_sensor_readings("sgp40", DAYS)
combined_df = merge_sensor_readings([bme280_df, veml7700_df, sgp40_df])
combined_df.head()

In [ ]:
# Generate a daily summary data frame
daily_df = combined_df.resample("D").agg({
    "temperature": ["min", "max", "mean", "median"],
    "humidity": ["min", "max", "mean", "median"],
    "pressure": ["min", "max", "mean", "median"],
    "illuminance": ["min", "max", "mean", "median"],
    "vocindex": ["min", "max", "mean", "median"],
    "issaturated": "sum",          # number of saturated readings
    "rating": lambda x: x.value_counts().index[0] if len(x) else None
})

daily_df.head()

In [ ]:
# Get the export folder path
export_folder_path = get_export_folder_path("daily")

# Strip the timezone from the timestamp, as this will cause the export to spreadsheet to fail (Excel can"t
# handle dates with timezone information)
daily_df.index = daily_df.index.tz_localize(None)

display(daily_df.head())

# Export the data to a spreadsheet
export_to_spreadsheet(export_folder_path, "bme280_temperature_daily_summary.xlsx", { "Daily Summary": daily_df["temperature"] })
export_to_spreadsheet(export_folder_path, "bme280_pressure_daily_summary.xlsx", { "Daily Summary": daily_df["pressure"] })
export_to_spreadsheet(export_folder_path, "bme280_humidity_daily_summary.xlsx", { "Daily Summary": daily_df["humidity"] })
export_to_spreadsheet(export_folder_path, "veml7700_illuminance_daily_summary.xlsx", { "Daily Summary": daily_df["illuminance"] })
export_to_spreadsheet(export_folder_path, "sgp40_vocindex_daily_summary.xlsx", { "Daily Summary": daily_df["vocindex"] })

# Daily Mean Temperature

In [ ]:
import matplotlib.pyplot as plt

degree_sign = u"\N{DEGREE SIGN}"

plt.figure(figsize=(12, 6))
plt.plot(daily_df["temperature"]["mean"], label="Mean Temperature")
plt.fill_between(
    daily_df.index,
    daily_df["temperature"]["min"],
    daily_df["temperature"]["max"],
    alpha=0.2,
    label="Range"
)

plt.title("Daily Temperature Range")
plt.ylabel(f"Temperature ({degree_sign}C)")
plt.legend()

# Export to PNG or PDF, if required
export_chart(export_folder_path, "bme280_temperature_daily_summary", "png")

plt.show()

# Daily Mean Pressure

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(daily_df["pressure"]["mean"], label="Mean Pressure")
plt.fill_between(
    daily_df.index,
    daily_df["pressure"]["min"],
    daily_df["pressure"]["max"],
    alpha=0.2,
    label="Range"
)

plt.title("Daily Pressure Range")
plt.ylabel("Pressure (hPa)")
plt.legend()

# Export to PNG or PDF, if required
export_chart(export_folder_path, "bme280_pressure_daily_summary", "png")

plt.show()

# Daily Mean Humidity

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(daily_df["humidity"]["mean"], label="Mean Humidity")
plt.fill_between(
    daily_df.index,
    daily_df["humidity"]["min"],
    daily_df["humidity"]["max"],
    alpha=0.2,
    label="Range"
)

plt.title("Daily Humidity Range")
plt.ylabel("Humidity (%)")
plt.legend()

# Export to PNG or PDF, if required
export_chart(export_folder_path, "bme280_humidity_daily_summary", "png")

plt.show()

# Daily Mean Illuminance

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(daily_df["illuminance"]["mean"], label="Mean Illuminance")
plt.fill_between(
    daily_df.index,
    daily_df["illuminance"]["min"],
    daily_df["illuminance"]["max"],
    alpha=0.2,
    label="Range"
)

plt.title("Daily Illuminance Range")
plt.ylabel("Illuminance (lux)")
plt.legend()

# Export to PNG or PDF, if required
export_chart(export_folder_path, "veml7700_illuminance_daily_summary", "png")

plt.show()

# Daily Mean VOC Index

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(daily_df["vocindex"]["mean"], label="Mean VOC Index")
plt.fill_between(
    daily_df.index,
    daily_df["vocindex"]["min"],
    daily_df["vocindex"]["max"],
    alpha=0.2,
    label="Range"
)

plt.title("Daily VOC Index Range")
plt.ylabel("VOC Index")
plt.legend()

# Export to PNG or PDF, if required
export_chart(export_folder_path, "sgp40_vocindex_daily_summary", "png")

plt.show()